In [1]:
# Set these variables according to your setup for the code below to use.
tesseract_executable = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
#importing all necessary libraries
import torch 
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pytesseract
import os
import pandas as pd
import re
from PIL import Image
from re import compile
import time
from datetime import datetime
import json
import requests

In [3]:
pytesseract.pytesseract.tesseract_cmd = tesseract_executable


In [4]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path = 'weights/epochs_100/last.pt')

Using cache found in C:\Users\Iliyan Tashinov/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-20 Python-3.11.2 torch-2.0.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [5]:
def get_extraction(gray_frame,detections,frame_count):
    
    frame_count = frame_count
    plate_format = re.compile('^[a-zA-Z]{2}[0-9]{4}[a-zA-Z]{2}$')
    plate_format1 = re.compile('^[a-zA-Z]{1}[0-9]{4}[a-zA-Z]{2}$')
    data = detections
    crop = get_crop(gray_frame,data)
    crop_contoured = get_contour1(crop)
    license_plate = extract_license(crop_contoured,frame_count)
    print(license_plate)
    
    if plate_format.match(license_plate) is not None or plate_format1.match(license_plate) is not None:
    
        return crop_contoured,license_plate
    
    license_plate = "N/A"
    return crop_contoured,license_plate


In [6]:
def get_crop(frame,detections):
    
    a,b = get_frameProportions(frame)
    data = detections
    crop = frame[int(detections[1]*a):int(detections[3]*a),
                 int(detections[0]*b):int(detections[2]*b)]
    cv2.imwrite(os.path.join('crop' ,'detection.jpg'), crop)
    return crop


In [7]:
def get_contour1(crop):

    threshold = cv2.threshold(crop, 85, 255,cv2.THRESH_BINARY)[1]
    crop_contoured = cv2.bitwise_not(threshold)
    
    
    return crop_contoured

In [8]:
def get_contour2(crop):

    ret, thresh = cv2.threshold(crop, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    img = cv2.blur(thresh, ksize=(1,1))
    kernelmatrix = np.ones((2, 1), np.uint8)
    blurred = cv2.dilate(img, kernelmatrix, iterations=1)
    ret, resultimage = cv2.threshold(thresh, 0, 255, cv2.THRESH_TRIANGLE)
    crop_contoured = cv2.erode(resultimage, kernelmatrix, iterations=2)
    
    
    return crop_contoured

In [9]:
def extract_license(crop_contoured,frame_count):

    analysis = cv2.connectedComponentsWithStats(crop_contoured,4,cv2.CV_32S)
    (totalLabels, label_ids, values, centroid) = analysis
    symbolList = []
    xCentroidList = []
    
    for i in range(1,totalLabels):
        
        area = values[i, cv2.CC_STAT_AREA] 
        X,Y = centroid[i]
        
        if (area < 50) or (area > 1000):
            continue
            
        new_img = crop_contoured.copy()

        x1 = values[i, cv2.CC_STAT_LEFT]
        y1 = values[i, cv2.CC_STAT_TOP]
        w = values[i, cv2.CC_STAT_WIDTH]
        h = values[i, cv2.CC_STAT_HEIGHT]

        #convert it to 255 value to mark it white
        component = (label_ids == i).astype("uint8") * 255

        # Increase brightness by multiplying with a scaling factor
        brightness_scale = 2  # Adjust this value to increase or decrease brightness
        component = component * brightness_scale
        cropped_component = component[y1-6:y1+h+6, x1-6:x1+w+6]
        #cropped_component = cv2.resize(cropped_component, (15, 15))
        #cv2.imshow("Individual Component", cropped_component)
        #cv2.waitKey(0)
        try:
            cv2.imwrite(os.path.join('component' , f'- frame - {frame_count} - component detection.jpg'), cropped_component) #saving crop file in component directory    
            cv2.imwrite(os.path.join('crop' , f'frame - {frame_count} - wholeframe.jpg'), crop_contoured)#saving crop file in crop directory
            symbol = pytesseract.image_to_string(cropped_component, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 10 --oem 3')
            symbol = symbol.replace('\n', '')
            print(symbol)
            symbolList.append(symbol)
            xCentroidList.append(X)

        except:
            continue
                
    print(symbolList)    
    xCentroidList, symbolList = zip(*sorted(zip(xCentroidList, symbolList)))
    license_plate = ''.join(symbolList)
            
    return license_plate

In [10]:
def format_frame(frame):
    
    resultimage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    return resultimage

In [11]:
 def check_license(license_plate):
        
    my_url = 'https://check.bgtoll.bg/check/vignette/plate/BG/' + license_plate
    x = requests.get(my_url)
    if (x.status_code != 200):
        result_text = "error"
        time_left = "error"
        return result_text, time_left
    
    p = json.loads(x.content)        
    
    if (p['vignette'] is None):
        result_text = "Vignette not valid."
        time_left = "N/A"
        return result_text,time_left
    
    result_text = "Vignette is valid"
    time_left = p['vignette']['validityDateToFormated']
    return result_text,time_left
        



In [12]:
def get_frameProportions(frame):
    a,b = frame.shape
    
    a_p = a/608
    b_p = b/608
    
    return a_p, b_p

In [13]:
def draw_rectangle(frame,detection):
    
        data = results.xyxy[0][i]
        a = (int(data[0]), int(data[1]))
        b = (int(data[2]), int(data[3]))
        
        cv2.rectangle(frame,a,b,(0,255,0), 3)
        
        return a, b

In [14]:
#set empty lists
crop_index = []
number_plate = []
date =[]
validation_df = []
time_remaining_lst = []

#define video source
video_1 = 'video_test.mp4'


#load video
cap = cv2.VideoCapture(video_1)
frame_count = 1


#start video
while cap.isOpened():
    
    
    ret, frame = cap.read() #ret is boolean, returns true if frame is available;
    frame_resized = cv2.resize(frame, (608, 608)) #resizing the frames for better utilization of YOLO
    
    results = model(frame_resized) #run YOLO on the resized frame
    
    print(f'-----number of plates detected = {len((results.xyxy[0]))} - at frame {frame_count}--------------')

    #    ---      ---      ---        0  1  2  3  4  5      
    #loop through elements in tensor[h1,w1,h2,w2,cf,lb] 
    for i in range(len(results.xyxy[0])): #len represents the number of tensors;n tensors is the number of detections;#the loop is executed for each tenso
                                           
        print(f'checking confidence level of detection - {i + 1} ')

        conf = results.xyxy[0][i][4] #take the ith tensor/detection ; #conf = data[4] #take the confidence of that detection
        
        if (conf < 0.80): #checking how confident is the model; not acceptable under 80%
            print("detection's confidence is too low")
            print(f'++++end of analysis for detection number++++ {i+1}')
            continue
            
        print(f'passed the confidence test')
        
        draw_rectangle(frame_resized,results.xyxy[0][i])
        gray_frame = format_frame(frame)
        extraction = get_extraction(gray_frame, results.xyxy[0][i],frame_count)
        text_license = extraction[1]
        
        if text_license == "N/A":
            continue

        #check if plate has already been processed
        if (text_license in number_plate):
            
            print(f'{text_license} is already recorded')
            plate_index = number_plate.index(text_license)
            #frame_resized = cv2.putText(frame_resized, f"{validation_df[plate_index]} || {time_remaining_lst[plate_index]}", (a[0],a[1]-30), cv2.FONT_HERSHEY_SIMPLEX,
            #1, (255,0,0), 1, cv2.LINE_AA)
            continue
    
        
        print('new license plate found')
        validation = check_license(text_license) #Validating sticker
        vignette_status = validation[0]
        time_remaining = validation[1]


        #frame_resized = cv2.putText(frame_resized, f"{vignette_status} || {time_remaining}", (a[0],a[1]-30), cv2.FONT_HERSHEY_SIMPLEX,
        #1, (255,0,0), 1, cv2.LINE_AA)
        print(f'data for detection number and license plate {i+1} => {text_license} has been collected')

        #gathering data
        number_plate.append(text_license) # add number_plate
        crop_index.append(frame_count) # record frame of detection
        date.append(pd.datetime.now().strftime("%d/%m/%Y %H:%M:%S")) #record time of detection
        validation_df.append(vignette_status) #add vignette status
        time_remaining_lst.append(time_remaining) #add remaining time of vignette


        print(f'++++end of analysis for detection number {i+1}++++')
        
    frame_count += 1
    
    #display frame
    cv2.imshow('YOLO', frame_resized)
    
    if cv2.waitKey(10) & 0xFF == ord('s'):
        break
        
cap.release()
cv2.destroyAllWindows()


#aggregate data
data = list(zip(crop_index,number_plate,date
                ,validation_df,time_remaining_lst))
df = pd.DataFrame(data,columns = ["frame","number plate","time of detection","sticker status","time until expiry"])

-----number of plates detected = 1 - at frame 1--------------
checking confidence level of detection - 1 
detection's confidence is too low
++++end of analysis for detection number++++ 1
-----number of plates detected = 1 - at frame 2--------------
checking confidence level of detection - 1 
detection's confidence is too low
++++end of analysis for detection number++++ 1
-----number of plates detected = 1 - at frame 3--------------
checking confidence level of detection - 1 
detection's confidence is too low
++++end of analysis for detection number++++ 1
-----number of plates detected = 1 - at frame 4--------------
checking confidence level of detection - 1 
passed the confidence test
B

9
9
4
K
I
2

['B', '', '9', '9', '4', 'K', 'I', '2', '']
KI2994B
-----number of plates detected = 2 - at frame 5--------------
checking confidence level of detection - 1 
passed the confidence test
9
4
B

K
I
2
9

['9', '4', 'B', '', 'K', 'I', '2', '9', '']
KI2994B
checking confidence level of detectio

In [14]:
df

,frame,number plate,time of detection,sticker status,time until expiry


In [15]:
! pip3 freeze

absl-py==1.4.0
anyio==3.6.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
async-generator==1.10
attrs==22.2.0
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.1
cachetools==5.3.0
certifi==2022.12.7
cffi==1.15.1
charset-normalizer==3.1.0
chart-studio==1.1.0
colorama==0.4.6
colorlover==0.3.0
comm==0.1.2
contourpy==1.0.7
cufflinks==0.17.3
cycler==0.11.0
Cython==0.29.34
debugpy==1.6.5
decorator==5.1.1
defusedxml==0.7.1
docopt==0.6.2
entrypoints==0.4
et-xmlfile==1.1.0
exceptiongroup==1.1.1
executing==1.2.0
fastjsonschema==2.16.2
filelock==3.11.0
fonttools==4.38.0
fqdn==1.5.1
gitdb==4.0.10
GitPython==3.1.31
google-auth==2.17.2
google-auth-oauthlib==1.0.0
grpcio==1.53.0
h11==0.14.0
idna==3.4
imageio==2.31.0
ipykernel==6.20.2
ipython==8.8.0
ipython-genutils==0.2.0
ipywidgets==8.0.4
isoduration==20.11.0
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.4.4
jupyter-events==0.6.3
jupyter_client==7.4.9
